This notebook holds all development of ICA related analysis.  
  
Road map:  
1) Run ICA on running only PSAbool data from open field day -1. Should be able to identify cells with similar place fields!  Sanity check.  
  
2) Run ICA on circa-freezing data only. Another sanity check. Create boolean of freezing times +/- 2 seconds. Should be able to recover cells I have already identified as freeze cells with freezing_analysis.MotionTuning class.  
  
3) Run ICA on non-filtered PSAbool data. Correlate different component activations with freezing and/or velocity to pull out the "freezing" ensembles on day 1.  
  
3a) Track these ensembles backward and forward one session to see how they develop. Should match single cell findings, but maybe we will see something interesting in the ANI vs. Control stuff?

In [1]:
# Import useful modules
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import matplotlib.gridspec as gridspec
import pandas as pd

In [2]:
# Import project specific modules and enable automatic reloading
import sys
from pathlib import Path
import os
eraser_path = Path(os.getcwd()).parent
reinstatement_path = eraser_path.parent / 'FearReinstatement'
sys.path.append(str(eraser_path))
sys.path.append(str(reinstatement_path))
%load_ext autoreload
%autoreload 2

from session_directory import find_eraser_directory as get_dir
import er_plot_functions as erp
import Placefields as pf
import freezing_analysis as fa
import helpers 
import eraser_reference as err

# Project save directory here
print_dir = '/Users/nkinsky/Documents/BU/Working/Eraser/plots'

# First try to recover cells with similar place fields using ICA as a sanity check.

In [9]:
# First define moving average function to smooth things into 1 second
def moving_average(arr, n=20):
    ret = np.cumsum(arr, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

def bin_array(arr, n=20):
    return np.add.reduceat(arr, np.arange(0, len(arr), n))


In [10]:
c = [1, 0, 0, 2, 0, 2, 3, 3, 3, 4]
print(moving_average(c, n=3))
print(bin_array(c, n=3))

[0.33333333 0.66666667 0.66666667 1.33333333 1.66666667 2.66666667
 3.         3.33333333]
[1 4 9 4]


In [12]:
from sklearn.datasets import load_digits
from sklearn.decomposition import FastICA

PF = pf.load_pf('Marble07', 'Open', -1)
PSAsmooth, PSAbin = [], []
for psa in PF.PSAboolrun:
    PSAsmooth.append(moving_average(psa)) # Create 1 second moving-average array
    PSAbin.append(bin_array(psa)) # Create 1 second non-overlapping bin array
PSAsmooth = np.asarray(PSAsmooth)
PSAbin = np.asarray(PSAbin)

# Run ICA on the two different PSAs
transformer_sm = FastICA(n_components=7, random_state=0)
transformer_bin = FastICA(n_components=7, random_state=0)
PSAsm_trans = transformer_sm.fit_transform(PSAsmooth.T)
PSAbin_trans = transformer_bin.fit_transform(PSAbin.T)


In [14]:
transformer_bin.mixing_.shape
#plt.hist(PSAbin_trans)

(672, 7)

In [15]:
transformer_bin.components_.shape

(7, 672)

In [16]:
PSAbin.shape

(672, 578)

In [ ]:
PSAb